In [ ]:
from vlnce_baselines.common.env_utils import construct_envs
from habitat import Config
from vlnce_baselines.config.default import get_config
from habitat_baselines.common.environments import get_env_class

#Import config and perform some manipulation
exp_config ='vlnce_baselines/config/paper_configs/transformer.yaml'
config = get_config(exp_config, None)
split = config.TASK_CONFIG.DATASET.SPLIT
config.defrost()
config.TASK_CONFIG.TASK.NDTW.SPLIT = split
config.TASK_CONFIG.TASK.SDTW.SPLIT = split

# if doing teacher forcing, don't switch the scene until it is complete
if config.DAGGER.P == 1.0:
    config.TASK_CONFIG.ENVIRONMENT.ITERATOR_OPTIONS.MAX_SCENE_REPEAT_STEPS = (
        -1
    )
config.freeze()
envs = construct_envs(config, get_env_class(config.ENV_NAME))
obs=envs.reset()

## Setup Actor Critic Agent

In [ ]:
import pytorch_lightning as pl
from vlnce_baselines.models.transformer_policy_lightning import TransformerEncDec
trainer = pl.Trainer()
EncDec = TransformerEncDec()
    trainer.fit(EncDec)

from vlnce_baselines.models.transformer_policy import TransformerPolicy
import torch

load_from_ckpt = False
device = (
    torch.device("cuda", config.TORCH_GPU_ID)
    if torch.cuda.is_available()
    else torch.device("cpu")
)
model_config = config.MODEL

model_config.defrost()
model_config.TORCH_GPU_ID = config.TORCH_GPU_ID
model_config.freeze()
actor_critic = TransformerPolicy(
                observation_space=envs.observation_spaces[0],
                action_space=envs.action_spaces[0],
                model_config=model_config,
                num_processes = config.NUM_PROCESSES,
                batch_size=config.DAGGER.BATCH_SIZE,
            )
actor_critic.to(device)

optimizer = torch.optim.Adam(actor_critic.parameters(), 
                                         lr=config.MODEL.TRANSFORMER.lr, 
                                         betas=(0.9, 0.98), eps=1e-9)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, 
                                                               patience=config.MODEL.TRANSFORMER.scheduler_patience, 
                                                               verbose=True, min_lr=1e-6)

if load_from_ckpt:
    ckpt_dict = load_checkpoint(ckpt_path, map_location="cpu")
    actor_critic.load_state_dict(ckpt_dict["state_dict"])
    
# if config.DAGGER.PRELOAD_LMDB_FEATURES:
#     envs.close()
#     del envs
#     envs = None